In [1]:
from Loader import fileloader as fl
from Loader import datavalidator as dv
from Loader import datahandledatabase as dhdb
import posixpath
import logging

logger = logging.getLogger("")
logger.setLevel(logging.INFO)

month = '201902'
inifile = fl.ReadIniFile()
defaultpath = inifile.getDataPath()
testpath = inifile.getTestPath()

INFO:    Values loaded from globals.ini:
INFO:     * mainpath D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel Peru S.A/MercadoEmpresas
INFO:     * scriptspath D:/Datos de Usuario/jsaenza/Documents/Comisiones/Comisiones-python
INFO:     * databasepath D:/Datos de Usuario/jsaenza/Documents/Comisiones/SQLiteDB
INFO:    datapath value is D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx
INFO:    testpath value is D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/test


In [2]:
parser = inifile.getIniFileParser()

loader = fl.LoadFileProcess(month)
loader.setParser(parser)
loader.setDefaultPath(defaultpath)

In [8]:
# Importando la información
# Validar en los equipos quincenales la jerarquia de ventas
# Copiar los reportes comisionales del periodo : Grandes Cuentas y Pymes

# Inar y eliminando las migraciones

#inarbruto = loader.loadFile('Inar_bruto')


#jerarquia = loader.loadFile('Jerarquia')

#comisionantes = loader.loadFile('Comisionantes_voz')


dbparser = inifile.getDbIniFileParser()

dbmanager = dhdb.DbDataProcess(month,mercado="empresas")
dbmanager.setParser(dbparser)
tblempleados = dbmanager.loadData('tblEmpleados')


INFO:    Setting database path to D:/Datos de Usuario/jsaenza/Documents/Comisiones/SQLiteDB/mercado_empresas_db.sqlite


Los registros de la tabla tblEmpleados es 1060 registros  


In [4]:
inarbruto.dtypes

CODIGO             object
RAZON_SOCIAL       object
CONTRATO            int64
FEC_ACTIV          object
FEC_DESACTIVA      object
MODELO             object
TELEFONO            int64
ESTADO             object
VENDEDOR           object
PLAN_TARIFARIO     object
ZONA               object
MOTIVO             object
DEALER             object
TMCODE              int64
FECHA_PROCESO      object
FALSO_DEAC         object
TECNOLOGIA         object
TIPODOC            object
DOCUMENTO          object
NRO_ORDEN         float64
dtype: object

In [9]:
# Armando las validaciones

valinarfl = {'inarbruto' : inarbruto, 'jerarquia' : jerarquia, 'comisionantes' : comisionantes, 'tblempleados' : tblempleados}
valinarobj = dv.ValidateInar(valinarfl)
inarvalidation = valinarobj.validation()

valmultipledatafl = {'jerarquia' : jerarquia, 'comisionantes' : comisionantes, 'tblempleados' : tblempleados}
valmultipledataobj = dv.ValidateMultipleData(valmultipledatafl)
multipledatavalidation = valmultipledataobj.validation()

#Exportando validaciones

xlsxfile1 = posixpath.join(testpath, month + '_Validaciones en INAR.xlsx')
exportparams = {'xlsxfile' : xlsxfile1, 'dataframe' : inarvalidation}
valinarobj.exportValidation(exportparams)

#"""
xlsxfile2 = posixpath.join(testpath, month + '_Otras Validaciones.xlsx')
exportparams = {'xlsxfile' : xlsxfile2, 'dataframe' : multipledatavalidation}
valmultipledataobj.exportValidation(exportparams)
#"""

D:\Datos de Usuario\jsaenza\Documents\Comisiones\Comisiones-python\Loader\datavalidator.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  blanksdoc['OBSERVACION'] = 'blanks en documento'
D:\Datos de Usuario\jsaenza\Documents\Comisiones\Comisiones-python\Loader\datavalidator.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  blanksvendedor['OBSERVACION'] = 'blanks en vendedor o zona'
C:\Users\jsaenza\anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future vers

Archivo exportado D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/test/201902_Validaciones en INAR.xlsx con 1034 registros
Archivo exportado D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/test/201902_Otras Validaciones.xlsx con 74 registros


In [ ]:
# Detectando los registros de Tienda que no tienen Orden
str_tienda = 'Tienda'
lst_estados = ['NEWS','ADDS','FCHURN','DEOF','REAC']

mask0 = inarbruto['ESTADO'].isin(lst_estados)
mask1 = inarbruto['NRO_ORDEN'].isnull()
inar_f1 = inarbruto[mask0 & mask1]
mask2 = inar_f1['VENDEDOR'].notnull()
inar_f2 = inar_f1[mask2]
mask3 = inar_f2['VENDEDOR'].str.contains(str_tienda)
inar_f3 = inar_f2[mask3]

#mask2 = inarbruto['VENDEDOR'].str.contains(str_tienda)
#inar_filtrado = inarbruto[mask2]



In [ ]:
inar_f2.to_csv(posixpath.join(testpath, 'inar_sin_orden.csv'))

In [ ]:
tblempleados[tblempleados['CODIGO_INAR'] == 'RPAZ2']

In [ ]:
# Archivos para Sherman
exportar = False
if exportar:
    df1 = inarbruto[:500000]
    df2 = inarbruto[500001:]
    df1.to_csv(testpath + month + '_sherman1.csv', index = False)
    df2.to_csv(testpath + month + '_sherman2.csv', index = False)

In [ ]:
inarbruto[inarbruto['CONTRATO'] == 43035266]
#inarbruto[inarbruto['codigo'] == '1.10108190']
#inarvalidation[inarvalidation['contrato'] == 37838662]

In [ ]:
tblempleados.dtypes